# Summer Institute in Computational Social Science 2025

In [1]:
import dspy

lm = dspy.LM('openai/gpt-4o', api_key='API_KEY')
dspy.configure(lm=lm)

In [2]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

In [3]:
example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Academy Award for Best Director', 'Chris Noonan', 'Miss Potter']


In [4]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-10 results."""

    topK = search(query, k=10)
    return topK

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [5]:
DOCS = {}

def search(query: str, k: int) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=k)
    results = [x['text'] for x in results]

    for result in results:
        title, text = result.split(" | ", 1)
        DOCS[title] = text

    return results

In [6]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

In [7]:
react(claim="David Gregory was born in 1625.").titles[:3]

['David Gregory (physician)']

In [8]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=10)

In [9]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

Average Metric: 79.00 / 100 (79.0%): 100%|████████████████████████████████████████████| 100/100 [01:46<00:00,  1.07s/it]

2025/01/08 10:25:16 INFO dspy.evaluate.evaluate: Average Metric: 78.99999999999999 / 100 (79.0%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Oxford Movement, Samuel Rickards, Trinity Episcopal Church (Hough...","{'thought_0': 'To verify the claim, I need to identify the specifi...",The claim states that the Church of England's movement that inspir...,"[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...",✔️ [1.000]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Red, White &amp; Crüe, Mike Tyson, Bobby Stewart]","{'thought_0': 'The claim mentions ""Red, White & Crüe"" and a French...","The claim suggests a connection between ""Red, White & Crüe"" and a ...","[Red, White & Crüe, Bobby Stewart, Mike Tyson]",✔️ [0.667]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Glen or Glenda, Fernand Rivers, Ed Wood]","{'thought_0': 'To verify the claim, I need to find information abo...","The claim states that the writer/director/actor from ""Glen or Glen...","[Ed Wood, Fernand Rivers, Glen or Glenda]",✔️ [1.000]
3,The film by Sandi Sissel was released before The End of Suburbia.,"[Chicken Ranch (film), The End of Suburbia, Sandi Sissel]","{'thought_0': 'To verify the claim, I need to find the release dat...","To verify the claim, I searched for films associated with Sandi Si...","[Salaam Bombay!, Chicken Ranch, The End of Suburbia]",✔️ [0.333]
4,The actor who played captain hook in the live production with Tayl...,"[Peter Pan Live!, Taylor Louderman, Christopher Walken]","{'thought_0': 'To verify this claim, I need to identify the actor ...",The claim states that the actor who played Captain Hook in the liv...,"[Christopher Walken, Peter Pan Live!, The Deer Hunter]",✔️ [0.667]
5,"The movie, based on a story written by brothers who are interred a...","[Alter St.-Matthäus-Kirchhof, The Princess and the Frog, The Frog ...","{'thought_0': 'To verify the claim, I need to identify the movie i...",The claim is that a movie based on a story written by brothers int...,"[The Brothers Grimm (film), Brothers Grimm, Iron Henry]",
6,Federated Auto Parts 400 is an annual Monster Energy NASCAR Cup Se...,"[Toyota Owners 400, Crown Royal, Federated Auto Parts 400]","{'thought_0': 'To verify the claim, I need to find information abo...",The claim states that the Federated Auto Parts 400 is an annual NA...,"[Federated Auto Parts 400, Toyota Owners 400, Crown Royal]",✔️ [1.000]
7,"The child actor, who played the character Fenmore Baldwin, plays i...","[Robbie Tucker, Fenmore Baldwin, It's Always Sunny in Philadelphia]","{'thought_0': 'First, I need to identify the child actor who playe...",The claim states that the child actor who played Fenmore Baldwin i...,"[Robbie Tucker, It's Always Sunny in Philadelphia, Fenmore Baldwin...",✔️ [1.000]
8,"An actor played the role of ""Annabelle"" in the film The Fruit Mach...","[The Fruit Machine, Rubeus Hagrid, Robbie Coltrane]","{'thought_0': 'The claim involves an actor who played ""Annabelle"" ...","The claim states that an actor played the role of ""Annabelle"" in t...","[The Fruit Machine (film), Robbie Coltrane, Harry Potter (film ser...",✔️ [0.333]
9,"The father of the founder of St Hugh's College, Oxford, was a bish...","[St Hugh's College, Oxford, Elizabeth Wordsworth, Christopher Word...","{'thought_0': ""To verify the claim, I need to identify the founder...",The claim states that the father of the founder of St Hugh's Colle...,"[Elizabeth Wordsworth, Christopher Wordsworth, St Hugh's College, ...",✔️ [1.000]


79.0